### 모듈 불러오기

In [2]:
from langchain.chat_models import ChatOpenAI  # ChatOpenAI 모델을 가져옵니다. OpenAI 기반의 챗봇 모델입니다.
from langchain.document_loaders import UnstructuredFileLoader  # 파일에서 데이터를 읽어오는 데 사용하는 로더를 가져옵니다.
from langchain.text_splitter import CharacterTextSplitter  # 텍스트를 특정 기준에 따라 나누는 도구를 가져옵니다.
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings  # 텍스트를 임베딩 벡터로 변환하는 도구를 가져옵니다.
from langchain.vectorstores import FAISS  # 효율적인 벡터 검색을 위한 FAISS 라이브러리를 가져옵니다.
from langchain.storage import LocalFileStore  # 로컬 파일 시스템을 데이터 저장소로 사용하는 클래스입니다.
from langchain.chains import RetrievalQA  # 검색 기반 질문 답변 체인을 위한 클래스입니다.



### text loader를 이용하여 file 불러오기

In [5]:

# ChatOpenAI 모델을 인스턴스화합니다. 이 모델은 OpenAI의 챗봇 API를 사용합니다.
llm = ChatOpenAI()

# 로컬 파일 캐시 저장소를 설정합니다. "./cache/" 경로에 캐시 데이터가 저장됩니다.
cache_dir = LocalFileStore("./cache/")

# 텍스트를 분할하는 설정을 정의합니다. TikToken 기반의 인코더를 사용하며, 
# 텍스트를 "\n"을 기준으로 나누고, 각 청크는 600자까지 허용하며, 100자의 중첩을 포함합니다.
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # 줄바꿈을 기준으로 텍스트를 나눕니다.
    chunk_size=600,  # 각 텍스트 조각의 최대 크기를 600자로 설정합니다.
    chunk_overlap=100,  # 텍스트 조각 간 100자의 중복을 유지합니다.
)

# UnstructuredFileLoader를 사용해 텍스트 파일 "document.txt"를 로드합니다.
# UnstructuredFileLoader으로 txt, doc, pdf, html, exel 등 파일을 불러올 수 있음
loader = UnstructuredFileLoader("./document.txt")   

### file 분할
* 긴 text를 다루기 위해서 
* document를 작게 분할하면, 작업이 쉬워지고 응답 속도 빨라지고, LLM 사요 비용이 저렴해지는 장점이 있다

<span style="color: skyblue;">cached_embeddings</span>
* document 별로 embedding 작업 후 결과를 정한 vectore store를 반환
* document 검색, 연관성 높은 document를 찾음


In [ ]:
docs = loader.load_and_split(text_splitter=splitter)

# embedding: text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현
# 임베딩 모델로 openaiembedding 모델 사용
embeddings = OpenAIEmbeddings()

# 만들어진 embedding을 cache로 저장
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# document, embedding, from_document 메서드 호출
vectorstore = FAISS.from_documents(docs, cached_embeddings)

<span style="color: skyblue;">retriever</span>
* Langchain이 제공하는 class 또는 interface의 일종
* document를 검색해서 찾아오는 기능을 제공


In [ ]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansins")